# Roteiro para importar dados para um banco Mongo a partir de JSON

  - This file is part of  program Alite.
  - Copyright © 2023  Carlo Oliveira <carlo@nce.ufrj.br>.
  - [Labase](http://labase.selfip.org) - [NCE](http://portal.nce.ufrj.br) - [UFRJ](http://www.ufrj.br).
  - SPDX-License-Identifier: ([GNU General Public License v3.0 or later](https://www.gnu.org/licenses/gpl-3.0-standalone.html)).

## Version and configurations.

 This module retrieves data and plot statistics for the [Alite - Games](https://games.alite.selfip.org).

## [23.08](https://github.com/labase/data_edu/releases)

### Added

- Lê o JSON e ativa o banco (02)


## Recebe dados do arquivo JSON e limpa

In [67]:
import pandas as pd
from json import loads
import re
# jfile = "/home/carlo/Documentos/doc/vault/backup/alite/score"
jfile = "/home/carlo/Documentos/doc/vault/backup/alite/score"
# jfile = "score.json"
jfileo = "scoreo.json"
with open(jfile, "r") as fin, open(jfileo, "w") as out:
    din = fin.read()
    regex = re.compile(r'\{"\$oid":"([a-f0-9]*)"\}', re.S)
    din = re.sub(regex, r'"\1"', din)
    din = din.replace("}\n{", "},\n{")
    din = din.replace("0x01[]", "name")
    din = din.replace("0x[]", "name")
    din = din.split("\n")
    din = "\n".join(din)
    din = f"[{din[:-1]}]"
    
    print(din[:300])
    out.write(din)
# df = load(jfileo)
dfj = loads(din)
df = pd.read_json(jfileo)
print(df)

[{"_id":"6477ce20f626d3cb95e08c8f","name":"admin","ano":"ano1","sexo":"masculino","idade":"anos10","time":"2023-05-31 19:45:52.561244-03:00","games":[{"game":"wcst","goal":0,"trial":0,"scorer":"6477ce52f626d3cb95e08c90"},{"game":"game","goal":0,"trial":0,"scorer":"6477cedcf626d3cb95e08c91"}]},
{"_id
                          _id             name   ano       sexo   idade  \
0    6477ce20f626d3cb95e08c8f            admin  ano1  masculino  anos10   
1    6477ce52f626d3cb95e08c90              NaN   NaN        NaN     NaN   
2    6477cedcf626d3cb95e08c91              NaN   NaN        NaN     NaN   
3    6477cf19f626d3cb95e08c92            outro  ano2   feminino   anos9   
4    6477cf1df626d3cb95e08c93              NaN   NaN        NaN     NaN   
..                        ...              ...   ...        ...     ...   
224  64ad8b55b23753d1d92bcc04          Willian  ano6  masculino   anos8   
225  64ad8b57b23753d1d92bcc05              NaN   NaN        NaN     NaN   
226  64ad900eb23753d1d92

## Classe para lidar com o MongoDB

In [90]:
import pymongo
from bson.objectid import ObjectId
from pymongo import UpdateOne
# host="mongodb://mongo:27017"
host="mongodb://localhost:27017"
# host="mongodb://172.17.0.4:49155"


def init(data_base="alite_game", collection="ascore", con_string=None):
    con_string = con_string or host
    client = pymongo.MongoClient(con_string)
    mydb = client[data_base]

    my_col = mydb[collection]
    print("Connected to collection:", my_col)
    return my_col


class Facade:
    def __init__(self, data_base="alite_game", collection="ascore", db=None):
        # con = options.dbcon
        self.db = db if db else init(data_base=data_base, collection=collection)

    def load_any(self):
        dbt = self.db.find()
        return [ob for ob in dbt]

    def load_all(self):
        dbt = self.db.find({"games": {"$exists": True}})
        return [ob for ob in dbt]

    def load_player(self, oid):
        oid = oid if oid is ObjectId else ObjectId(oid)
        dbt = self.db.find_one(filter={"_id": oid})  # or self.db.find_one()
        # print("load_player dbt", oid, dbt)
        return dbt or dict(_id=f"THIS ID: {oid} WAS NOT FOUND")

    def load_item(self, item_dict):
        if item_dict and "_id" in item_dict:
            item_dict["_id"] = id_ if (id_ := item_dict["_id"]) is ObjectId else ObjectId(id_)
        dbt = self.db.find_one(filter=item_dict) if item_dict else self.db.find_one()
        return dbt

    def save_all(self, items, idx="oid", _idx="_id"):
        # _ = [dt.pop(_idx) for dt in items if _idx in items]
        _ids_data = [(dt.pop(_idx), dt) for dt in items if _idx in dt]
        operations = [UpdateOne({idx: idn}, {'$set': data_}, upsert=True) for idn, data_ in _ids_data]
        # operations = [UpdateOne({_item}, {'$set': data_}, upsert=True) for _item in items]
        self.db.bulk_write(operations)

    def insert(self, items):
        def oider(_oid):
            # return  ObjectId(_oid)
            return  _oid
        oid = oider(items["_id"])
        if "games" in items:
            _games = items["games"]
            items["games"] = [{k: v if k != "scorer" else oider(v) for k, v in gm.items()} for gm in _games]
            
        items.update({"_id": oid})
        print("items", items)
        ids = self.db.insert_one(items)
        return ids.inserted_id


## Insere dados no MongoDB

In [91]:
fac = Facade()
dat = dfj
print("dat",dat)
# print("inserted:", fac.insert(dat))
print("inserted:", fac.save_all(dat))

Connected to collection: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'alite_game'), 'ascore')
dat [{'name': 'admin', 'ano': 'ano1', 'sexo': 'masculino', 'idade': 'anos10', 'time': '2023-05-31 19:45:52.561244-03:00', 'games': [{'game': 'wcst', 'goal': 0, 'trial': 0, 'scorer': ObjectId('6477ce52f626d3cb95e08c90')}, {'game': 'game', 'goal': 0, 'trial': 0, 'scorer': ObjectId('6477cedcf626d3cb95e08c91')}]}, {'score': [{'carta': 0, 'casa': 'no no no', 'move': 'no no no', 'ponto': '000', 'valor': '0000', 'time': '2023-05-31 19:46:43.649000'}, {'carta': 1, 'casa': 'Duas Estrelas Verdes', 'move': 'Um Triângulo Verde', 'ponto': '100', 'valor': '1000', 'time': '2023-05-31 19:46:49.652000'}]}, {'score': [{'carta': '__I_N_I_T__', 'casa': '0_0', 'move': 'ok', 'ponto': '_INIT_', 'valor': True, 'time': '2023-05-31 19:49:01.943000'}, {'carta': '__A_T_I_V_A__', 'casa': '0_0', 'move': 'ok', 'ponto': '_CHAVES_', 'valor': True, 'time': '2023